In [1]:
import numpy as np
import pandas as pd
import random
import math 
import copy 
import sys

In [2]:
class wolf:
    def __init__(self, fitness, dim,seed, lb, ub):
        self.features = [0 for i in range(dim)]
        self.rand = random.Random(seed)
        self.position = [0.0 for i in range(dim)]
        for i in range(dim):
            self.position[i] = ((ub - lb) * self.rand.random() + lb)
        self.fitness = np.NINF

In [3]:
# rastrigin function
def fitness_rastrigin(position):
    fitness_value = 0.0
    for i in range(len(position)):
        xi = position[i]
        fitness_value += (xi * xi) - (10 * math.cos(2 * math.pi * xi)) + 10
    return fitness_value

In [4]:
#sphere function
def fitness_sphere(position):
    fitness_value = 0.0
    for i in range(len(position)):
        xi = position[i]
        fitness_value += (xi*xi)
    return fitness_value

In [5]:
def stepc(A, Di):
    try:
        c = 1/(1+ math.exp(10*(A*Di-0.5)))
    except Exception as e:
        return 1
    else:
        return c

In [6]:
def stepb(A, Di, r):
    if stepc(A,Di) >= r:
        return 1
    else:
        return 0

In [7]:
def X_d(X, A, Di, rand):
    r = rand.random()
    if (X + stepb(A, Di, r)) >= 1:
        return 1
    else:
        return 0

In [8]:
def cross_over(X_d1, X_d2, X_d3, rand):
    r = rand.random()
    if r < 1/3:
        return X_d1
    elif r >= 1/3 and r < 2/3:
        return X_d2
    else:
        return X_d3

In [9]:
def GWO(X, fitness, D, N, max_iter, lb, ub):
    rand = random.Random()
    population = [ wolf(X, fitness, D, i, lb, ub) for i in range(N)]
    population = sorted(population, key = lambda x : x.fitness)
    alpha_wolf, beta_wolf, gamma_wolf = copy.copy(population[:3])

    iteration = 0
    while iteration<max_iter:                  
        b = 2*(1 - iteration/max_iter)

        for i in range(N):    
            
            A1, A2, A3 = b * (2 * rand.random() - 1), b * (
                2 * rand.random() - 1), b * (2 * rand.random() - 1)
            C1, C2, C3 = 2*rand.random(), 2*rand.random(), 2*rand.random()
            
            X1 = [0.0 for i in range(D)]
            X2 = [0.0 for i in range(D)]
            X3 = [0.0 for i in range(D)]
            Xnew = [0.0 for i in range(D)]
            
            X_d1 = [0.0 for i in range(D)]
            X_d2 = [0.0 for i in range(D)]
            X_d3 = [0.0 for i in range(D)]
            X_dnew = [0.0 for i in range(D)]

            for j in range(D):
                
                D_alpha = abs(C1 * alpha_wolf.position[j] - population[i].position[j])
                D_beta = abs(C2 *  beta_wolf.position[j] - population[i].position[j])
                D_gamma = abs(C3 * gamma_wolf.position[j] - population[i].position[j])
                
                X1[j] = alpha_wolf.position[j] - A1 * D_alpha
                X2[j] = beta_wolf.position[j] - A2 * D_beta
                X3[j] = gamma_wolf.position[j] - A3 * D_gamma
                Xnew[j] = (X1[j] + X2[j] + X3[j]) /3.0  
                
                X_d1[j] = X_d(X1[j], A1, D_alpha, rand)
                X_d2[j] = X_d(X2[j], A2, D_beta, rand)
                X_d3[j] = X_d(X3[j], A3, D_gamma, rand)
                X_dnew[j] = cross_over(X_d1[j], X_d2[j], X_d3[j], rand) 
            
  
            fnew = fitness(Xnew)
            if fnew > population[i].fitness:
                population[i].position = Xnew
                population[i].fitness = fnew
                population[i].features = X_dnew
 
                 
        population = sorted(population, key = lambda x: x.fitness)
        alpha_wolf, beta_wolf, gamma_wolf = copy.copy(population[:3])
        iteration+=1
        
    print("Iteration = {} best fitness = {} ".format(str(iteration), alpha_wolf.fitness))
    return alpha_wolf.features